In [139]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
import plotly.express as px
import emd
tf.keras.mixed_precision.set_global_policy('mixed_float16')

BATCH_SIZE = 32
CHANNEL_NUMBER = 3
WINDOW_SIZE = 300
SLIDING_STEP = int(WINDOW_SIZE * 0.4)
KEY_CLASS = {0:'undefined action', 1:'up', 2:'down', 3:'left', 4:'right'}
CLASS_NUMBER = 3
NUM_IMF = 5
UNDEFINED_ACTION = 0
LABEL_THRESHOLD = 0.7

if CLASS_NUMBER < 2:
    CLASS_NUMBER = 2

def slicingAndNormalize(arr):
    def normalizer(X):
        mean = np.average(X, axis=1)
        std = np.std(X, axis=1)
        ret = (X.copy() - mean) / std
        return ret
    
    totalLength = arr.shape[0]
    if totalLength <= WINDOW_SIZE:
        return arr
    ret = normalizer((arr[:WINDOW_SIZE, :])[np.newaxis, :])
    
    i = SLIDING_STEP
    while (totalLength - i) > WINDOW_SIZE:
        new = normalizer((arr[i:(i + WINDOW_SIZE), :])[np.newaxis, :])
        ret = np.concatenate([ret, new], axis=0)
        i += SLIDING_STEP
    return ret

def slicing(x, y):
    totalLength = x.shape[0]
    assert totalLength == y.shape[0], "Data numbers not matching with that of labels."
    if totalLength <= WINDOW_SIZE:
        return x
    
    thresholdWindow = LABEL_THRESHOLD * WINDOW_SIZE
    retx = (x[:WINDOW_SIZE, :])[np.newaxis, :]
    rety = [np.argmax(np.sum(y[:WINDOW_SIZE], axis=0))]
    
    i = SLIDING_STEP
    while (totalLength - i) > WINDOW_SIZE:
        new = (x[i:(i + WINDOW_SIZE), :])[np.newaxis, :]
        retx = np.concatenate([retx, new], axis=0)
        classSum = np.sum(y[i:(i + WINDOW_SIZE)], axis = 0)
        maxIdx = np.argmax(classSum)
        if classSum[maxIdx] > thresholdWindow:
            rety.append(maxIdx)
        else:
            rety.append(UNDEFINED_ACTION)
        i += SLIDING_STEP
    return retx, rety

def one_hot_label(arr):
    label_dict = {}
    index = 1
    for val in arr:
        if not label_dict.get(val, False):
            label_dict[val] = index
            index += 1
    return label_dict

def reverse_label(label_dict):
    reverse_label_dict = {}
    for key in label_dict:
        reverse_label_dict[label_dict[key] - 1] = KEY_CLASS[key]
    return reverse_label_dict

def one_hot(arr, label_dict = None):
    if not isinstance(label_dict, dict):
        label_dict = one_hot_label(arr)

    ret = []
    key_num = len(list(label_dict.keys()))
    for val in arr:
        tmp = [0] * key_num
        tmp[label_dict[val] - 1] = 1
        ret.append(np.array(tmp))
        
    return np.array(ret)    

def emdSignal(sig):
    dataNumber = sig.shape[0]
    channel = sig.shape[-1]
    ret = None
    
    for i in range(dataNumber):
        temp = None
        
        for c in range(channel):
            raw = sig[i, :, c]
            imf = emd.sift.sift(raw, max_imfs=NUM_IMF, imf_opts={'sd_thresh': 0.1})
            
            if imf.shape[-1] < NUM_IMF:
                compensate = np.zeros((WINDOW_SIZE, NUM_IMF - imf.shape[-1]))
                imf = np.concatenate([imf, compensate], axis = 1)
            
            if not type(temp) == np.ndarray: 
                temp = imf
            else: 
                temp = np.concatenate([temp, imf], axis = 1)
            
        if type(temp) == np.ndarray: 
            if not type(ret) == np.ndarray: 
                ret = temp[np.newaxis, :]
            else: 
                ret = np.concatenate([ret, temp[np.newaxis, :]], axis = 0)
                
    return ret

def buildModel(shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape = shape),
        tf.keras.layers.Conv1D(80, 100, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(axis=1),
        tf.keras.layers.MaxPool1D(padding='same'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv1D(100, 120, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(axis=1),
        tf.keras.layers.MaxPool1D(padding='same'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv1D(120, 140, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(axis=1),
        tf.keras.layers.MaxPool1D(padding='same'),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(CLASS_NUMBER, activation='softmax')]
    )
    model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=[tf.keras.metrics.CategoricalAccuracy()])
                        #  tf.keras.metrics.Precision(thresholds = 0.5),
                        #  tf.keras.metrics.Recall(thresholds= 0.5)])
    model.summary()
    return model

In [32]:
trainFile = './data/train_X.npy'
trainLabelFile = "./data/train_y.npy"
testFile = './data/test1_X.npy'
testLabelFile = "./data/test1_y.npy"

trainSignal = np.load(trainFile)
trainLabel = np.load(trainLabelFile)
testSignal = np.load(testFile)
testLabel = np.load(testLabelFile)

In [ ]:
label_dict = one_hot_label(trainLabel)
reverse_label_dict = reverse_label(label_dict)
X_all, y_all = slicing(np.concatenate([trainSignal, testSignal], axis=0),
                       one_hot(np.concatenate([trainLabel, testLabel], axis=0), label_dict))
X = emdSignal(X_all)
y = one_hot(y_all)

print(X.shape)
print(label_dict)
print(reverse_label_dict)

In [39]:
train_label_dict = one_hot_label(trainLabel)
X, y = slicing(trainSignal, one_hot(trainLabel, train_label_dict))
train_out_label_dict = one_hot_label(y)
y = one_hot(y, train_out_label_dict)
X = emdSignal(X)

X_test, y_test = slicing(testSignal, one_hot(testLabel, train_label_dict))
y_test = one_hot(y_test, train_out_label_dict)
X_test = emdSignal(X_test)
print(train_label_dict)
print(X.shape)
print(X_test.shape)

{0: 1, 4: 2, 3: 3}
(496, 300, 15)
(279, 300, 15)


In [ ]:
threeChan = X[18]
f = threeChan[:, 0].T
s = threeChan[:, 1].T
t = threeChan[:, 2].T
imf = emd.sift.sift(f, max_imfs=4, imf_opts={'sd_thresh': 0.1})
print(imf.shape)
emd.plotting.plot_imfs(imf)

In [ ]:
n += 1
print(n, n*SLIDING_STEP, n*SLIDING_STEP + WINDOW_SIZE)
px.line(X[n]).show()

In [130]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777)

In [140]:
model = buildModel(X.shape[1:])
history = model.fit(x=X_train,
                    y=y_train,
                    batch_size=BATCH_SIZE,
                    epochs=100,
                    validation_data=(X_test,y_test))

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_63 (Conv1D)          (None, 300, 80)           120080    
                                                                 
 batch_normalization_63 (Bat  (None, 300, 80)          1200      
 chNormalization)                                                
                                                                 
 max_pooling1d_63 (MaxPoolin  (None, 150, 80)          0         
 g1D)                                                            
                                                                 
 dropout_44 (Dropout)        (None, 150, 80)           0         
                                                                 
 conv1d_64 (Conv1D)          (None, 150, 100)          960100    
                                                                 
 batch_normalization_64 (Bat  (None, 150, 100)       

In [126]:
model = tf.keras.models.load_model('./model/LickingPark')

In [141]:
res = model.predict(X)

25/25 [==============================] - 0s 5ms/step


In [126]:
pd.DataFrame([reverse_label_dict[np.argmax(x)] for x in y]).value_counts()

undefined action    526
left                131
right               120
dtype: int64

In [142]:
pd.DataFrame([reverse_label_dict[np.argmax(x)] for x in res]).value_counts()

undefined action    503
left                157
right               117
dtype: int64

In [47]:
model.save("./model/LickingPark", save_format="tf")

In [145]:
f1 = np.array(history.history['loss']).flatten()
valf1 = np.array(history.history['val_loss']).flatten()
px.line(pd.DataFrame(np.array([f1, valf1]).T, columns=['loss', 'val_loss'])).show()